In [ ]:
# This is an attempt to solve the screening task to find numbers that are less than a reference value using Quantum programming
# This task is a part of skills assessment for the QOSF's Quantum Computing Mentorship Program - cohort 9
# Programming this in the Quantum simulator provided by IBMQ the open source platform that I am most familiar with

In [57]:
# Install and import the relevant qiskit packages
%pip install qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import IntegerComparator
from qiskit_aer import Aer

Note: you may need to restart the kernel to use updated packages.


In [58]:
# instantiate the AER simulator to run the circuit
backend=Aer.get_backend('aer_simulator')

In [59]:
# Array to hold the boolean representation of the number to be compared    
vectorbit=[]
# Array to hold the result register value for the numbers compared
marksmallnum=[]
# Array to hold the list of small numbers identified
smallnumlist=[]

In [60]:
# Function to consttuct the quantum circuit to compare the two integers
# This function takes the constant integer to be compared and the bit string representation of the number from the list.
# A quantum circuit is built using the IntegerComparator function 
def construct_circuit_and_compare(k,bitstr):
    
    # reverse the bitstring since IBMQ is little endian 
    revstr=bitstr[::-1]
    l=len(bitstr)
    
    # we need one qubit each to represent one bit, so construct a Quantum register with as many qubits
    q = QuantumRegister(l,'q')
    # One classical register to measure the output
    c = ClassicalRegister(1,'c')

    circuit = QuantumCircuit(q,c)
  
    # Loading classical information to qubit by applying Pauli X gate to set the qubit to 1 where the classical bit is 1
    for i in range(l):
        if revstr[i]=='1':
            circuit.x(q[i])
            
    # Use IntegerComparator to compare the two numbers - This operation is based on two's complement implementation of binary subtraction but only uses carry bits and no actual result bits. 
    # The geq flag controls the interpretation of the result if the most significant carry bit (the results bit) is 1, the ≥ condition is True otherwise it is False
    
    comparator = IntegerComparator(num_state_qubits=2, value=k, geq=True)
    circuit = circuit.compose(comparator)
    circuit.measure(q[l-1],c[0])

    print(circuit)
    # run the circuit 
    new_circuit = transpile(circuit, backend)
    job = backend.run(new_circuit)
    counts = job.result().get_counts()

    
    # Read register value to extract the result
    Answer = [(k[::-1],v) for k,v in counts.items()]
    Answer.sort(key = lambda x: x[1], reverse=True)
    Y = []
    for k, v in Answer: Y.append( [ int(c) for c in k ] )
    #Add the result register value to a reference array
    marksmallnum.append(Y[0][0])
   
    #reset the job and counts parameters and the buffer used to store the reverse of the binary string for the number
    job=""
    counts=""
    revstr=""

In [61]:
# Custom function that accepts an integer 'k' and a list of integers that need to be compared against k to determine the ones that are smaller than k
# This function converts the integers in the list to their boolean representations and then calls a secondary function that is used to compare the integers using a quantum circuit
# The state of the result (which marks a 0 if the number is less than k) read from the quantum circuit is used at the end to identify the smaller numbers from the list and print the result set

def less_than_k(k,list):
    print(k)

    
    n=len(list)
    #print("list length ",n)
    
        
    vector=np.asarray(list, dtype=int)
   
    m=len('{0:b}'.format(max(vector)))
    
    #print("vectorbit len =",m)
    
    vectorbit=[]
    #convert to boolen representation of the numbers in the list
    for item in list:
        vectorbit.append('{0:0{size}b}'.format(item, size=m))
    
    for i in range(n):
        #print("i = ",i)
        print("vectorbit[i] = ",vectorbit[i])
        construct_circuit_and_compare(k,vectorbit[i])
    
    #get length of the array containing identified numbers less than k
    #smallnumarrlen=len(marksmallnum)
    
    #print("smallnumarrlen = ",smallnumarrlen)

    #construct result array
    if len(marksmallnum) > 0:
        indices = [index for (index, item) in enumerate(marksmallnum) if item == 0]
        for j in indices:
            smallnumlist.append(list[j])
            
    # print the original list for reference
    print(" original list ", k , list)
    # display result array    
    print(" numbers less than ",k," are ",smallnumlist)

In [62]:
# Test Data set
less_than_k(7,[4,9,11,14,1,13,6,15])
#less_than_k(8,[4,9,12,5,7,13,15,6,3])

# reset the placeholder vectors to run the next iteration of comparison
vectorbit=[]
marksmallnum=[]
smallnumlist=[]

7
vectorbit[i] =  0100
          ┌──────┐   
q_0: ─────┤0     ├───
          │      │   
q_1: ─────┤1     ├───
     ┌───┐│  cmp │   
q_2: ┤ X ├┤2     ├───
     └───┘│      │┌─┐
q_3: ─────┤3     ├┤M├
          └──────┘└╥┘
c: 1/══════════════╩═
                   0 
vectorbit[i] =  1001
     ┌───┐┌──────┐   
q_0: ┤ X ├┤0     ├───
     └───┘│      │   
q_1: ─────┤1     ├───
          │  cmp │   
q_2: ─────┤2     ├───
     ┌───┐│      │┌─┐
q_3: ┤ X ├┤3     ├┤M├
     └───┘└──────┘└╥┘
c: 1/══════════════╩═
                   0 
vectorbit[i] =  1011
     ┌───┐┌──────┐   
q_0: ┤ X ├┤0     ├───
     ├───┤│      │   
q_1: ┤ X ├┤1     ├───
     └───┘│  cmp │   
q_2: ─────┤2     ├───
     ┌───┐│      │┌─┐
q_3: ┤ X ├┤3     ├┤M├
     └───┘└──────┘└╥┘
c: 1/══════════════╩═
                   0 
vectorbit[i] =  1110
          ┌──────┐   
q_0: ─────┤0     ├───
     ┌───┐│      │   
q_1: ┤ X ├┤1     ├───
     ├───┤│  cmp │   
q_2: ┤ X ├┤2     ├───
     ├───┤│      │┌─┐
q_3: ┤ X ├┤3     ├┤M├
     └───┘└─

In [ ]:
# References 
# [1] Deutsch, David, and Richard Jozsa. "Rapid solution of problems by quantum computation." Proceedings of the Royal Society of London. Series A: Mathematical and Physical Sciences 439.1907 (1992): 553-558.
# [2] Bernstein, Ethan, and Umesh Vazirani. "Quantum complexity theory." SIAM Journal on computing 26.5 (1997): 1411-1473.
# [3] Grover, Lov K. , "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (1996), arXiv:quant-ph/9605043
# [4] Qiskit Documentation - https://docs.quantum.ibm.com/api/qiskit/0.26/
# [5] John A. Cortese, Timothy M. Braje "Loading Classical Data into a Quantum Computer"
# [6] Rubens Viana Ramos, Paulo Benício Melo de Sousa and David Sena Oliveira, "Solving mathematical problems with quantum search algorithm"
# [7] Ashish Mandoi, "Quantum Search on an unstructured input -  https://github.com/AsishMandoi/quantum-search"